In [37]:
import pandas as pd
import numpy as np
import difflib as dl
import multiprocessing
import nltk
# nltk.download('stopwords')
from os import cpu_count
from unidecode import unidecode
from tqdm import tqdm
tqdm.pandas(desc="my bar!")
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk.stem.lancaster import LancasterStemmer

In [17]:
year = 2013
df = pd.read_csv(f'../datasets/raw/{year}.csv', sep=';', encoding ='latin')
df = df[['DS_RESUMO', 'DS_PALAVRA_CHAVE', 'NM_AREA_AVALIACAO']]

In [18]:
df.columns = df.columns.str.slice(3).str.lower()
df.rename(columns={
    'base': 'ano'
}, inplace=True)

for item in list(df.iloc[:, 0:].columns):
    df[f'{item}'] = df[f'{item}'].str.lower()

In [19]:
def rename_areas(text: str) -> str:
    if text in ['administração pública e de empresas, ciências contábeis e turismo', 'administração, ciências contábeis e turismo']:
        return 'administração pública e de empresas, ciências contábeis e turismo'
    elif text in ['arquitetura e urbanismo', 'arquitetura, urbanismo e design']:
        return 'arquitetura, urbanismo e design'
    elif text in 'filosofia/teologia:subcomissão filosofia':
        return 'filosofia'
    elif text in ['filosofia/teologia:subcomissão teologia', 'teologia']:
        return 'ciências da religião e teologia'
    elif text in 'letras / linguística':
        return 'linguística e literatura'
    elif text in 'artes / música':
        return 'artes'
    elif text in 'ciências sociais aplicadas i':
        return 'serviço social'
    else:
        return text

In [20]:
df.area_avaliacao = df.area_avaliacao.apply(rename_areas)
del rename_areas

In [21]:
colegios_dict = {
    'ciências da vida': {
        'ciências agrárias': ['ciência de alimentos', 'ciências agrárias i', 'medicina veterinária', 'zootecnia / recursos pesqueiros'],
        'ciências biológicas': ['biodiversidade', 'ciências biológicas i', 'ciências biológicas ii', 'ciências biológicas iii'],
        'ciências da saúde': ['educação física', 'enfermagem', 'farmácia', 'medicina i', 'medicina ii', 'medicina iii', 'nutrição', 'odontologia', 'saúde coletiva']
    },
    'humanidades': {
        'ciências humanas': ['antropologia / arqueologia', 'ciência política e relações internacionais', 'ciências da religião e teologia', 'educação', 'filosofia', 'geografia', 'história', 'psicologia', 'sociologia'],
        'ciências sociais aplicadas': ['administração pública e de empresas, ciências contábeis e turismo', 'arquitetura, urbanismo e design', 'comunicação e informação', 'direito', 'economia', 'planejamento urbano e regional / demografia', 'serviço social'],
        'linguística, letras e artes': ['artes', 'linguística e literatura']
    },
    'ciências exatas, tecnológicas e multidisciplinar': {
        'ciências exatas e da terra': ['astronomia / física', 'ciência da computação', 'geociências', 'matemática / probabilidade e estatística', 'química'],
        'engenharias': ['engenharias i', 'engenharias ii', 'engenharias iii', 'engenharias iv'],
        'multidisciplinar': ['biotecnologia', 'ciências ambientais', 'ensino', 'interdisciplinar', 'materiais']
    }
}

In [22]:
df['colegios'] = ''
df['areas'] = ''
df.rename(columns={'area_avaliacao': 'subareas'}, inplace=True)

for colegios, areas in colegios_dict.items():
    for areas, subareas in areas.items():
        df.loc[df.subareas.isin(subareas), 'colegios'] = colegios
        df.loc[df.subareas.isin(subareas), 'areas'] = areas
        
df = df[['resumo', 'palavra_chave', 'colegios', 'areas', 'subareas']]
del colegios_dict

In [23]:
df.palavra_chave.replace(to_replace = r'1[.]', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' \d[.]', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'(\S{2,})(?:\.)', value = '\\1;', regex=True, inplace=True)
df.palavra_chave.replace(to_replace = r'\b(pt|en)\b', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' ,|, | [,] | /|/ | [/] ', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' [.] |\(\d\)', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' \x96 |\x93', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'[^a-z0-9\s\x80-\xff,.:;?!-]', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r' - |; | ;|; ;', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r';{2,}', value = ';', regex = True, inplace=True)
df.palavra_chave.replace(to_replace= r'[\W_]+$|^[;]+', value='', regex=True, inplace=True)
df.palavra_chave.replace(to_replace = r'palavras-chave\S', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'^\s+|\s+$|\x80', value = '', regex = True, inplace=True)
df.palavra_chave.replace(to_replace = r'[^\w\s;]', value = '', regex = True, inplace=True)

In [24]:
df.resumo.replace(to_replace= r'[^a-zA-Z0-9\s]+', value='', regex=True, inplace=True)

In [25]:
def remove_stopwords(text: str) -> str:
    stop_words = set(stopwords.words('portuguese'))
    new_text = [item for item in wordpunct_tokenize(text) if item not in stop_words]
    return " ".join(new_text)

In [26]:
def lc_stem(text: str) -> str:
    stemmer = LancasterStemmer()
    words = [stemmer.stem(word) for word in wordpunct_tokenize(text)]
    return " ".join(words)

In [27]:
df.resumo = df.resumo.progress_apply(remove_stopwords).progress_apply(lc_stem).progress_apply(lambda x: unidecode(x))
df.palavra_chave = df.palavra_chave.progress_apply(remove_stopwords).progress_apply(lc_stem).progress_apply(lambda x: unidecode(x))
df.palavra_chave.replace(to_replace = r' ; ', value = ';', regex = True, inplace=True)
del remove_stopwords, lc_stem

my bar!: 100%|██████████| 67534/67534 [00:00<00:00, 149190.25it/s]


In [28]:
df.drop(df.query("palavra_chave == '' | resumo == ''").index, inplace=True)
df.reset_index(drop=True, inplace=True)

In [29]:
key_words = df.palavra_chave.copy()
key_words = list(set(';'.join(key_words).split(';')))

key_words = [item.replace(r'[^a-zA-Z0-9\s]+', '') for item in key_words]

key_words = [item for item in key_words if 2 < len(item) < 80]
key_words = [item for item in key_words if not item.isdigit()]
for i, _ in enumerate(key_words):
    key_words[i] = key_words[i].strip()

key_words = list(set(key_words))

In [30]:
test = [item.replace(' ', '') for item in key_words]

test = list(set(test))

In [31]:
len(test)

108773

In [32]:
len(key_words)

109190

In [33]:
def calculate_similarity_or_pertency(sentence: str, text: str) -> str:
    if sentence == '' or text == '':
        return False
    if sentence in text or text in sentence:
        return True
    # similaridade = dl.SequenceMatcher(lambda x: x == " ", sentence, text).real_quick_ratio()
    similaridade = dl.SequenceMatcher(lambda x: x == " ", sentence, text).ratio()
    porcentagem = int(similaridade * 100)
    if porcentagem > 75:
        return True
    
    return False

In [34]:
def apply_similarity(row, character):
    chave = calculate_similarity_or_pertency(character, row['palavra_chave'])
    resumo = calculate_similarity_or_pertency(character, row['resumo'])
    return chave or resumo

- 50 palavras_chave = 1m 1.3s
- 500 palavras_chave = 9 24.4s
- 5000 palavras_chave = 126m 2.3s
- 109190 palavras_chave = ?

In [ ]:
def process_character(character):    
    cut = df[df.apply(lambda row: apply_similarity(row, character), axis=1)].iloc[:, -5:]
    results = []
    for subarea in sorted(cut.subareas.unique()):
        new_cut = cut[cut.subareas == subarea]
        colegio, area = new_cut.colegios.unique()[0], new_cut.areas.unique()[0]
        frequencia = new_cut.palavra_chave.str.count(character).sum() + new_cut.resumo.str.count(character).sum()
        if frequencia == 0: frequencia = new_cut.subareas.value_counts()[0]
        results.append([colegio, area, subarea, character, frequencia])
    
    return results

max_threads = cpu_count()
data = []
with multiprocessing.Pool(processes=max_threads) as pool:
    for result in pool.imap_unordered(process_character, key_words[:50]):
        data.extend(result)

In [ ]:
columns_dtype = {
    'colegio': 'category',
    'area': 'category',
    'subarea': 'category',
    'palavra_chave': 'category',
    'frequencia': 'uint64'
}

freq = pd.DataFrame(data, columns=columns_dtype.keys()).astype(columns_dtype)

In [ ]:
freq.set_index('palavra_chave').sort_index().sort_values('frequencia', ascending=False)

,colegio,area,subarea,frequencia
palavra_chave,,,,
stic,humanidades,"linguística, letras e artes",linguística e literatura,4477
desenvolv,humanidades,ciências humanas,educação,4002
diverso problema pertin class npdificil pres mundo real,humanidades,ciências humanas,educação,3604
cristianismo heresiolog retoric tertuliano valentinianismo,humanidades,ciências humanas,educação,3587
sistem renin angiotensin aldosteron remodelamento vascul,humanidades,ciências humanas,educação,3580
...,...,...,...,...
naturez luz,humanidades,ciências sociais aplicadas,comunicação e informação,1
invest perform,humanidades,ciências sociais aplicadas,"administração pública e de empresas, ciências ...",1
naturez luz,ciências da vida,ciências da saúde,nutrição,1
